In [1]:
from analogainas.search_spaces.config_space import ConfigSpace

In [2]:
## Default Search Space
CS = ConfigSpace(search_space="resnet-like",dataset='ImageNet')

In [3]:
CS.get_hyperparameters()

['out_channel0', 'M', 'R1', 'R2', 'R3', 'R4', 'R5', 'convblock1', 'convblock2', 'convblock3', 'convblock4', 'convblock5', 'widenfact1', 'widenfact2', 'widenfact3', 'widenfact4', 'widenfact5', 'B1', 'B2', 'B3', 'B4', 'B5']


In [4]:
CS.compute_cs_size()

9516645089280

In [5]:
# Sample possible configurations
configs = CS.sample_arch_uniformly(5) # sample 5 configs

## Evaluator 

To speed up the search, we built a machine learning predictor to evaluate the accuracy and robustness of any given architecture from the configuration space. 

In [6]:
# Load the evaluator 
from analogainas.evaluators.xgboost import XGBoostEvaluator
evaluator = XGBoostEvaluator()

In [7]:
# The ranker model ranks the architectures according to their 1-day accuracy. It is trained with a listwise training loss. 
evaluator.ranker 

XGBRegressor(base_score='8.5658985E-1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
# The AVM predictor regresses the average monthly variation. 
evaluator.avm_predictor

XGBRegressor(base_score='9.32594E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Search Optimizer and Worker 
In this example, we will use evolutionary search to look for the best architecture in CS using our evaluator. 

In [9]:
from analogainas.search_algorithms.ea_optimized import EAOptimizer
from analogainas.search_algorithms.worker import Worker


In [10]:
optimizer = EAOptimizer(evaluator, population_size=20, nb_iter=10)  

In [11]:
NB_RUN = 1
worker = Worker(CS, optimizer=optimizer, runs=NB_RUN)

In [12]:
worker.search()

The 'results' directory already exists.

Search 0 started
2
ITERATION 0 completed: best acc [0.7990944]
ITERATION 1 completed: best acc [0.7990944]
ITERATION 2 completed: best acc [0.7990944]
ITERATION 3 completed: best acc [0.8845372]
ITERATION 4 completed: best acc [0.92043924]
ITERATION 5 completed: best acc [0.9240905]
ITERATION 6 completed: best acc [0.9240905]
ITERATION 7 completed: best acc [0.9240905]
ITERATION 8 completed: best acc [0.9240905]
ITERATION 9 completed: best acc [0.9240905]
Best Acc = [0.9240905]
SEARCH ENDED


/home/research/.conda/envs/analognas/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/research/.conda/envs/analognas/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
worker.result_summary()

Best architecture accuracy:  [0.9240905]
Standard deviation of accuracy over 1 runs: nan
Best architecture:  {'out_channel0': 32, 'M': 4, 'R1': 3, 'R2': 5, 'R3': 12, 'R4': 10, 'R5': 0, 'convblock1': 1, 'convblock2': 2, 'convblock3': 1, 'convblock4': 2, 'convblock5': 0, 'widenfact1': 0.5127229713200394, 'widenfact2': 0.7163255341265284, 'widenfact3': 0.6197214949382625, 'widenfact4': 0.5486812092842295, 'widenfact5': 0, 'B1': 6, 'B2': 2, 'B3': 1, 'B4': 3, 'B5': 0}


In [14]:
worker.best_config

{'out_channel0': 32,
 'M': 4,
 'R1': 3,
 'R2': 5,
 'R3': 12,
 'R4': 10,
 'R5': 0,
 'convblock1': 1,
 'convblock2': 2,
 'convblock3': 1,
 'convblock4': 2,
 'convblock5': 0,
 'widenfact1': 0.5127229713200394,
 'widenfact2': 0.7163255341265284,
 'widenfact3': 0.6197214949382625,
 'widenfact4': 0.5486812092842295,
 'widenfact5': 0,
 'B1': 6,
 'B2': 2,
 'B3': 1,
 'B4': 3,
 'B5': 0}

In [15]:
from analogainas.search_spaces.resnet_macro_architecture import ImageNetNetwork

In [16]:
test_net = ImageNetNetwork(worker.best_config)
total_params = sum(p.numel() for p in test_net.parameters())
print("Total number of parameters:", total_params)

Total number of parameters: 40099115
